In [96]:
import os
import re
import json
import numpy as np
import pandas as pd

# ***Code Function***

***Extract the data from JSON files and save it in a CSV file as mobile_data.csv***

In [97]:
# Input and output directories
input_directory = './Jsons'
output_file = './mobile_data.csv'

# Define the column headers for the output CSV
columns = [
    "name", "brand", "2G", "3G", "4G", "5G", "Announced", "Status", "Weight", "Length", "Width", 
    "Diameter", "SIM", "Display Type", "Display Size", "PPI", "Body Ratio", "OS", "Battery_capacity", 
    "Price", "CPU", "Ratio", "Pixel", "WLAN", "Colors", "Sensors", "Bluetooth", "GPU", 
    "LoudSpeaker", "3.5mm jack", "Chipset", "Network", "Internal", "Card slot", "RAM", "Storage"
]

data = []

def extract_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = json.load(file)
        
        for top_key, models in content.items():
            brand_name = ''.join(filter(str.isalpha, top_key))
            
            for model, specs in models.items():                     # Extract various specifications
                dimensions = specs.get("Body", [{}])[0].get("Dimensions", "")
                dimensions_split = dimensions.split(' x ') if dimensions else []

                price = next((item.get("Price", "") for item in specs.get("Misc", []) if "Price" in item), "")

                chipset = next((item.get("Chipset", "") for item in specs.get("Platform", []) if "Chipset" in item), "")
                cpu = next((item.get("CPU", "") for item in specs.get("Platform", []) if "CPU" in item), "")
                gpu = next((item.get("GPU", "") for item in specs.get("Platform", []) if "GPU" in item), "")
                gpu_from_cpu = re.search(r',\s*(.*GPU)', cpu)
                if gpu_from_cpu:
                    gpu_info = gpu_from_cpu.group(1).rsplit(' GPU', 1)[0]
                    if not gpu:
                        gpu = gpu_info
                    cpu = re.sub(r',\s*.*GPU', '', cpu)

                cpu = re.sub(r'\s*\(.*\)', '', cpu)
                cpu = re.sub(r'(\d+(?:\.\d+)?)\s*GHz', r'\1 GHz', cpu)
                cpu = re.sub(r'(\d+(?:\.\d+)?)\s*MHz', r'\1 MHz', cpu)

                loudspeaker = specs.get("Sound", [{}])[0].get("Loudspeaker", "") == "Yes" if len(specs.get("Sound", [])) > 0 else False
                jack_3_5mm = specs.get("Sound", [{}])[1].get("3.5mm jack", "") == "Yes" if len(specs.get("Sound", [])) > 1 else False

                ram = next((item.get("Internal", "") for item in specs.get("Memory", []) if "RAM" in item.get("Internal", "")), "")
                storage = next((item.get("Internal", "") for item in specs.get("Memory", []) if "Internal" in item), "")
                memory = specs.get("Memory", [{}])
                internal_memory = next((item.get("Internal", "") for item in memory if "Internal" in item), "")
                card_slot = memory[0].get("Card slot", "")

                status = specs.get("Launch", [{}])[1].get("Status", "")
                status = 'Available' if isinstance(status, str) and status.lower().startswith('available') else (
                         'Coming' if isinstance(status, str) and status.lower().startswith('coming') else (
                         'discontinued' if pd.notna(status) else status))
                
                announced = specs.get("Launch", [{}])[0].get("Announced", "")
                month_map = {
                    'Jan': 'January', 'Feb': 'February', 'Mar': 'March', 'Apr': 'April',
                    'May': 'May', 'Jun': 'June', 'Jul': 'July', 'Aug': 'August',
                    'Sep': 'September', 'Oct': 'October', 'Nov': 'November', 'Dec': 'December'
                }

                announced_search = re.search(r'(\d{4}),\s*(\w+)\.?(\s+Released.*)?|(\d{4}),\s*(\w+)\s+(\d{1,2})', announced)

                if announced_search:
                    if announced_search.group(1):
                        year = announced_search.group(1)
                        month = announced_search.group(2)
                        announced = f"{month_map.get(month[:3], month)} {year}"
                    elif announced_search.group(4):
                        year = announced_search.group(4)
                        month = announced_search.group(5)
                        day = announced_search.group(6)
                        announced = f"{month_map.get(month[:3], month)} {day}, {year}"
                else:
                    announced_search = re.search(r'(\d{4})', announced)
                    announced = announced_search.group(1) if announced_search else None

                sim = specs.get("Body", [{}])[2].get("SIM", "").lower().strip()
                sim_options = ['nano-sim', 'micro-sim', 'mini-sim', 'esim']
                sim = ', '.join([s for s in sim_options if s in sim]) if sim else sim
                sim = 'mini-sim' if 'yes' in sim else sim

                ppi = specs.get("Display", [{}])[2].get("Resolution", "")
                ppi = re.search(r'~?(\d+)\s*(?:ppi|ppi density)', ppi)
                ppi = float(ppi.group(1)) if ppi else None

                diameter = dimensions_split[2] if len(dimensions_split) > 2 else ""
                diameter = re.search(r'(\d+(?:\.\d+)?)\s*mm', diameter)
                diameter = float(diameter.group(1)) if diameter else None

                weight = specs.get("Body", [{}])[1].get("Weight", "")
                weight = re.search(r'(\d+(?:\.\d+)?)\s*g', weight)
                weight = float(weight.group(1)) if weight else None

                display_type = specs.get("Display", [{}])[0].get("Type", "").split(',')[0].strip()

                display_size = specs.get("Display", [{}])[1].get("Size", "")
                display_size = re.search(r'(\d+(?:\.\d+)?)\s*inches', display_size)
                display_size = float(display_size.group(1)) if display_size else None

                body_ratio = specs.get("Display", [{}])[1].get("Size", "")
                body_ratio = re.search(r'(\d+(?:\.\d+)?)%', body_ratio.split("(~")[-1] if "(~" in body_ratio else "")
                body_ratio = float(body_ratio.group(1)) / 100 if body_ratio else None

                ratio = specs.get("Display", [{}])[2].get("Resolution", "")
                ratio = re.search(r', (\d+:\d+) ratio', ratio)
                ratio = ratio.group(1) if ratio else None

                os_info = specs.get("Platform", [{}])[0].get("OS", "")
                os_info = re.search(r'(Android|iOS|Windows|HarmonyOS|KaiOS|Tizen|Proprietary OS|Other OS)\s?(\d+(\.\d+)?)?', os_info, re.IGNORECASE)
                os_info = f"{os_info.group(1)} {os_info.group(2)}" if os_info and os_info.group(2) else os_info.group(1) if os_info else None

                battery = specs.get("Battery", [{}])[0].get("Type", "")
                battery_capacity = re.search(r'(\d+(?:\.\d+)?)\s*(mAh|Wh)', battery)
                if battery_capacity:
                    capacity, unit = battery_capacity.groups()
                    capacity = float(capacity)
                    if unit == 'Wh':
                        capacity = float(capacity) * 1000 / 3.7
                    battery_capacity = f"{round(capacity)} mAh"
                else:
                    battery_capacity = None

                pixel = specs.get("Display", [{}])[2].get("Resolution", "")
                pixel = re.search(r'(\d+\s*x\s*\d+)', pixel)
                pixel = pixel.group(1) if pixel else None

                ram = re.findall(r'(\d+(?:GB|MB|KB))\s*(?=RAM)', ram)
                ram = ', '.join(sorted(set(ram)))

                storage = re.findall(r'(\d+(?:GB|MB|KB))(?:(?=\s*\d+(?:GB|MB|KB))|(?=\s*ROM)|$)', storage)
                storage = ', '.join(sorted(set(storage)))

                network_generation_map = {
                    '2G': ['GSM', 'CDMA'],
                    '3G': ['UMTS', 'HSPA', 'EVDO'],
                    '4G': ['LTE'],
                    '5G': ['5G']
                }

                network_tech = specs.get("Network", [{}])[0].get("Technology", "").upper()
                
                # Compile row data
                row = {
                    "name": model,
                    "brand": brand_name,
                    "2G": any(tech in network_tech for tech in network_generation_map['2G']),
                    "3G": any(tech in network_tech for tech in network_generation_map['3G']),
                    "4G": any(tech in network_tech for tech in network_generation_map['4G']),
                    "5G": any(tech in network_tech for tech in network_generation_map['5G']),
                    "Announced": announced,
                    "Status": status,
                    "Weight": weight,
                    "Length": dimensions_split[0] if len(dimensions_split) > 0 else "",
                    "Width": dimensions_split[1] if len(dimensions_split) > 1 else "",
                    "Diameter": diameter,
                    "SIM": sim,
                    "Display Type": display_type,
                    "Display Size": display_size,
                    "PPI": ppi,
                    "Body Ratio": body_ratio,
                    "OS": os_info,
                    "Battery_capacity": battery_capacity,
                    "Price": price,
                    "CPU": cpu.strip() if cpu else None,
                    "Ratio": ratio,
                    "Pixel": pixel,
                    "WLAN": specs.get("Comms", [{}])[0].get("WLAN", ""),
                    "Colors": specs.get("Misc", [{}])[0].get("Colors", ""),
                    "Sensors": specs.get("Features", [{}])[0].get("Sensors", ""),
                    "Bluetooth": specs.get("Comms", [{}])[1].get("Bluetooth", ""),
                    "GPU": gpu.strip() if gpu else None,
                    "LoudSpeaker": loudspeaker,
                    "3.5mm jack": jack_3_5mm,
                    "Chipset": chipset,
                    "Network": specs.get("Network", [{}])[0].get("Technology", ""),
                    "Internal": internal_memory,
                    "Card slot": card_slot,
                    "RAM": ram,
                    "Storage": storage
                }
                data.append(row)

# Process each JSON file
for file_name in os.listdir(input_directory):
    if file_name.endswith(".json"):
        extract_data(os.path.join(input_directory, file_name))

# Convert to DataFrame and save to CSV
df = pd.DataFrame(data, columns=columns)
df.to_csv(output_file, index=False, encoding='utf-8-sig')